***2.3 Assignment - protobuf***

**Protobuf schema (book.proto)**

In [1]:
%%writefile ../schema/book.proto
syntax = "proto3";

message Book {
  string title = 1;
  string author = 2;
  int32 page_count = 3;
}

service Library {
  rpc checkoutBook(Book) returns (Book) {}
}

Writing ../schema/book.proto


**Protobuf server (book_server.py)**

In [ ]:
python -m grpc_tools.protoc -I./schema --python_out=. --grpc_python_out=. ./schema/book.proto
#run this in terminal to generate the book_pb2.py and book_pb2_grpc.py files

In [ ]:
%%writefile ../book_server.py
from concurrent import futures
import grpc

#Generated gPRC code from the book.proto file
import book_pb2
import book_pb2_grpc

class Library(book_pb2_grpc.LibraryServicer):

    def checkoutBook(self, request, context):
        # request is a Book message sent from the client
        # The question says: return the same Book message
        return request

#create the gRPC server and add the Library service to it
server = grpc.server(futures.ThreadPoolExecutor(max_workers=2))
#Register the Library service with the server
book_pb2_grpc.add_LibraryServicer_to_server(Library(), server)
server.add_insecure_port('[::]:50051')
print("Starting and running the server...")
print("Press Ctrl+C to stop the server.")
#start the server and keep it running until interrupted
server.start()
server.wait_for_termination()

Writing ../book_server.py


In [ ]:
# run this in terminal to start the server
python book_server.py

**Protobuf client (book_client.py)**

In [9]:
import sys
sys.path.append('..') #allow importing generated files from project root

import grpc
import book_pb2
import book_pb2_grpc

In [7]:
#Helper function to call the checkoutBook method on the server
def checkout_book(stub, book):

    #call the RPC method and returns a book
    book = stub.checkoutBook(book)
    return book

with grpc.insecure_channel('localhost:50051') as channel:
    # Create a Book message to send to the server
    my_book = book_pb2.Book(title="DSAI", author="ElaineT", page_count=96)

    #Create a stub (client) for the Library service
    stub = book_pb2_grpc.LibraryStub(channel)

    #Call the checkoutBook method and print the returned Book message
    returned = checkout_book(stub, my_book)

    # print result to show it is returned correctly
    print(returned)

title: "DSAI"
author: "ElaineT"
page_count: 96

